In [4]:
n = 95341235345618011251857577682324351171197688101180707030749869409235726634345899397258784261937590128088284421816891826202978052640992678267974129629670862991769812330793126662251062120518795878693122854189330426777286315442926939843468730196970939951374889986320771714519309125434348512571864406646232154103
e = 3
c = 63476139027102349822147098087901756023488558030079225358836870725611623045683759473454129221778690683914555720975250395929721681009556415292257804239149809875424000027362678341633901036035522299395660255954384685936351041718040558055860508481512479599089561391846007771856837130233678763953257086620228436828

In [5]:
c1 = (c * inverse_mod((2^456)^3, n)) % n

In [30]:
def trimZero(f):
    while len(f) > 0 and f[-1] == 0:
        f.pop()
    return f

def fillZero(f, lenFill):
    assert len(f) <= lenFill
    return f + [0] * (lenFill - len(f))
        
def modPoly(f, modulus):
    r = f.copy()
    for i in range(len(f)):
        r[i] %= modulus
    return trimZero(r)

def makeMonic(f, modulus):
    r = f.copy()
    leadCoefficientInverse = inverse_mod(r[-1], modulus)
    for i in range(len(r)):
        r[i] *= leadCoefficientInverse
        r[i] %= modulus
    return r

def addPoly(f1, f2):
    f3 = [0] * max(len(f1), len(f2))
    for i in range(len(f3)):
        if i < min(len(f1), len(f2)):
            f3[i] = f1[i] + f2[i]
        elif len(f1) < len(f2):
            f3[i] = f2[i]
        else:
            f3[i] = f1[i]
    return trimZero(f3)

def mulPoly(f1, f2):
    f3 = [0] * (len(f1) + len(f2))
    for i2 in range(len(f2)):
        for i1 in range(len(f1)):
            f3[i1 + i2] += f1[i1] * f2[i2]
    return trimZero(f3)

def powPoly(f, p):
    r = [1]
    for i in range(p):
        r = mulPoly(r, f)
    return r

def getRoots(f):
#     print("[i] Finding roots of {}...".format(f))
    _x_ = PolynomialRing(RationalField(), 'x').gen()
    fInPolyRep = 0
    for i in range(len(f)):
        fInPolyRep += f[i] * _x_ ^ i
    try:
        return fInPolyRep.roots()
    except:
        return []
    
def genPoly(f):
    _x_ = PolynomialRing(RationalField(), 'x').gen()
    fInPolyRep = 0
    for i in range(len(f)):
        fInPolyRep += f[i] * _x_ ^ i
    return fInPolyRep

def coppersmith(f, modulus, m, X=-1):    
    # Assure monic
    assert f[-1] == 1
    
    # Get degree & set X if necessary
    degree = len(f) - 1
    assert degree >= 1
    if X == -1:
        X = int(modulus ^ (1 / degree))
        if degree == 1:
            X -= 1
#         print('[No upperbound is set so the algorithm set it to {}]'.format(X))
    
    print("[] Solving {} = 0 mod {} \n\tfor x <= {} \n\tfor m = {}".format(genPoly(f), modulus, X, m))
    
    # Get dim
    dimension = (degree) * (m + 1)

    # Constructs a matrix.
    M = []
    for v in range( 0, m+1 ):
        for u in range( 0, degree ):
            # Generate g(x) function from f(x)
            g = mulPoly( mulPoly( [modulus^(m-v)], [0]*u+[1] ), powPoly(f, v) )
            
            # Create g(xX) from g(x)
            for i in range(len(g)):
                g[i] *= X^i
            
            # Append to row
            M.append( fillZero(g, dimension) )
    
    # Convert to matrix
    M = matrix(M)
#     print("[] M:\n")
#     print(M)
    
    # Assert if our matrix meets the criteria
#     det = 1
#     for i in range(dimension):
#         det *= M[i, i]
#     assert int((det ^ (1/dimension)) * (2 ^ (dimension/4))) < int(modulus ^ m / isqrt(dimension))
    
    # Apply LLL
    M = M.LLL()
#     print("\n[] M.LLL():\n")
#     print(M)
    
    
    # Scroll through the shortest vectors as the new polynomials
    roots = []
    for row in M:
        f = trimZero([int(col) for col in row])
        for i in range(len(f)):
            assert f[i] % (X^i) == 0
            f[i] //= X^i
#         print("[] f: {}, row: {}".format(f, row))
        roots += getRoots(f)
    
    # Return unique X component of roots
    return list(dict.fromkeys([root[0] for root in roots]))

In [31]:
br = ord('}')
f = mulPoly([(br^3 - c1) % n, 3 * br^2 * 2^8, 3 * br * 2^16, 2^24], [inverse_mod(2^24, n)])
f = modPoly(f, n)
coppersmith(f, n, 10)

[] Solving x^3 + 47298190972240185269476220178340596088836353081445116378536068027238036260007536029108850004945601352606297349885723679405383643302367461484502790870969529687323305335979402680101112848851121392945416415945488141409044383051764536562970815371153552241502386829151320342749813511445946332408698357984654232701*x^2 + 23467246511232973320064004296776948680764616497194610368545931143804649449747218371233847792500538121890138488896610749030765365099362989702464152298831492476199933513000853365917539719636363164050274120497559953526983727331693335977401643170277988164606145482930608650618148429906943906498668590448156293913*x + 57999478643836474067339126693368064839037979297955548145001486286807919797755998252395500875117791125412828060393509556919219356187321576498095043650785790252532978425915153390596259248391960617433395229196200571140609997509562395724991836789638851662285811282909313047681854778835192349650251623537863126664 = 0 mod 95341235345618011251857577682324351171

[54378971213289398953784088432450365616857947331656976856187769983595603428029791430301072236126565740]

In [14]:
coppersmith([5, 1], 11, 3, 10)

[] Solving x + 5 = 0 mod 11 
	for x <= 10 
	for m = 3
[] f: [-150, -35, 4, 1], row: (-150, -350, 400, 1000)
[] f: [-330, -11, 11], row: (-330, -110, 1100, 0)
[] f: [1331], row: (1331, 0, 0, 0)
[] f: [605, 121], row: (605, 1210, 0, 0)


[(6, 1), (-5, 2), (6, 1), (-5, 1), (-5, 1)]

In [23]:
from Crypto.Util.number import bytes_to_long, long_to_bytes
long_to_bytes(54378971213289398953784088432450365616857947331656976856187769983595603428029791430301072236126565740)

b'crypto{n0n_574nd4rd_p4d_c0n51d3r3d_h4rmful'